In [1]:
#set adequate flag for Theano on lxplus
import theano
theano.config.gcc.cxxflags = '-march=corei7'

#load needed things
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from keras.models import Sequential, Model
from keras.optimizers import SGD
from keras.layers import Input, Activation, Dense, Convolution2D, MaxPooling2D, Dropout, Flatten
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pyp
import ROOT
import itertools
import math

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

Using Theano backend.


Welcome to JupyROOT 6.07/07


In [2]:
#format the inputs from TTree
def formatInputs(files, njets, min_m4l, max_m4l):    
    formated_inputs = {}
    formated_inputs['VBF'] = []
    formated_inputs['GGH'] = []
    formated_inputs['ttH'] = []
    formated_inputs['ZH'] = []
    formated_inputs['WH'] = []
    formated_inputs['ggZZ'] = []
    formated_inputs['qqZZ'] = []
    formated_inputs['TTWJets'] = []
    formated_inputs['TTZ'] = []
    formated_inputs['ZZZ'] = []
    formated_inputs['WWZ'] = []
    formated_inputs['WZZ'] = []
    formated_inputs['WZ'] = []
    formated_inputs['WW'] = []
    formated_inputs['DYJets'] = []
    formated_inputs['TTbar'] = []
    formated_inputs['WJets'] = []

    for ifile in range(len(files)):
        file_name = files[ifile]
        tfile = ROOT.TFile.Open(file_name)
        tree = tfile.Get('HZZ4LeptonsAnalysisReduced')
        
        nacepted = 0
        for ievt, evt in enumerate(tree):
            if(evt.f_njets_pass >= njets and evt.f_mass4l >= min_m4l and evt.f_mass4l <= max_m4l): #VBF region
                event = []
                        
                lep = []
                lep.append( evt.f_lept1_pt )
                lep.append( evt.f_lept1_eta )
                lep.append( evt.f_lept1_phi )
                lep.append( evt.f_lept1_pdgid )
                event.append(lep)

                lep = []
                lep.append( evt.f_lept2_pt )
                lep.append( evt.f_lept2_eta )
                lep.append( evt.f_lept2_phi )
                lep.append( evt.f_lept2_pdgid )
                event.append(lep)

                lep = []
                lep.append( evt.f_lept3_pt )
                lep.append( evt.f_lept3_eta )
                lep.append( evt.f_lept3_phi )
                lep.append( evt.f_lept3_pdgid )
                event.append(lep)
    
                lep = []
                lep.append( evt.f_lept4_pt )
                lep.append( evt.f_lept4_eta )
                lep.append( evt.f_lept4_phi )
                lep.append( evt.f_lept4_pdgid )
                event.append(lep)
            
                jet = []
                jet.append( evt.f_jet1_highpt_pt )
                jet.append( evt.f_jet1_highpt_eta )
                jet.append( evt.f_jet1_highpt_phi )
                jet.append( evt.f_jet1_highpt_e*(ROOT.TMath.CosH(evt.f_jet1_highpt_eta)) )
                event.append(jet)

                jet = []
                jet.append( evt.f_jet2_highpt_pt )
                jet.append( evt.f_jet2_highpt_eta )
                jet.append( evt.f_jet2_highpt_phi )
                jet.append( evt.f_jet2_highpt_e*(ROOT.TMath.CosH(evt.f_jet2_highpt_eta)) )
                event.append(jet)

                jet = []
                jet.append( evt.f_jet3_highpt_pt )
                jet.append( evt.f_jet3_highpt_eta )
                jet.append( evt.f_jet3_highpt_phi )
                jet.append( evt.f_jet3_highpt_e*(ROOT.TMath.CosH(evt.f_jet3_highpt_eta)) )
                event.append(jet)
                
                event.append(evt.f_D_jet) #CMS VBF discriminant
                event.append(evt.f_Djet_VAJHU) #MELA
                event.append(evt.f_weight) #event weight
                
                event.append(evt.f_mass4l)
                event.append(evt.f_Z1mass)
                event.append(evt.f_Z2mass)                
                #nacepted += 1
                
                
                if(file_name.find('output_VBF') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['VBF'].append(event)
                    nacepted += 1
                if(file_name.find('output_GluGluH') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['GGH'].append(event)
                    nacepted += 1
                if(file_name.find('output_ttH') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['ttH'].append(event)
                    nacepted += 1
                if(file_name.find('output_ZH') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['ZH'].append(event)
                    nacepted += 1
                if(file_name.find('output_WminusH') != -1 or file_name.find('output_WplusH') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['WH'].append(event)
                    nacepted += 1
                if(file_name.find('output_GluGluToContinToZZ') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['ggZZ'].append(event)
                    nacepted += 1
                if(file_name.find('output_ZZTo4L') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['qqZZ'].append(event)
                    nacepted += 1
                if(file_name.find('output_TTWJets') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['TTWJets'].append(event)
                    nacepted += 1
                if(file_name.find('output_TTZ') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['TTZ'].append(event)
                    nacepted += 1
                if(file_name.find('output_ZZZ') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['ZZZ'].append(event)
                    nacepted += 1
                if(file_name.find('output_WWZ') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['WWZ'].append(event)
                    nacepted += 1
                if(file_name.find('output_WZZ') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['WZZ'].append(event)
                    nacepted += 1
                if(file_name.find('output_WZ') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['WZ'].append(event)
                    nacepted += 1
                if(file_name.find('output_WW') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['WW'].append(event)
                    nacepted += 1
                if(file_name.find('output_DYJets') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['DYJets'].append(event)
                    nacepted += 1
                if(file_name.find('output_TTTo') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['TTbar'].append(event)
                    nacepted += 1                            
                if(file_name.find('output_WJets') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['WJets'].append(event)
                    nacepted += 1


        #print ('%i - Processed (%i)' % (ifile, nacepted)), file_name

    return formated_inputs

In [3]:
#loads input data
file_names2e2mu = open('/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/histos2e2mu_25ns/filelist_2e2mu_2016_Spring16_AN_Bari_miniaod_mc.txt','r')
file_names4e = open('/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/histos4e_25ns/filelist_4e_2016_Spring16_AN_Bari_miniaod_mc.txt','r')
file_names4mu = open('/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/histos4mu_25ns/filelist_4mu_2016_Spring16_AN_Bari_miniaod_mc.txt','r')
path = '/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/'

files_addresses = []
for i in file_names2e2mu.readlines():
    files_addresses.append( path+'histos2e2mu_25ns/'+i.rstrip() )
for i in file_names4e.readlines():
    files_addresses.append( path+'histos4e_25ns/'+i.rstrip()  )
for i in file_names4mu.readlines():
    files_addresses.append( path+'histos4mu_25ns/'+i.rstrip()  )

#print files_addresses
events = formatInputs(files_addresses, 2, 118, 130)

('Processing', '/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/histos2e2mu_25ns/output_GluGluHToZZTo4L_M125_13TeV_powheg2_JHUgenV6_pythia8.root')
('Processing', '/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/histos2e2mu_25ns/output_ttH_HToZZ_4LFilter_M125_13TeV_powheg2_JHUgenV6_pythia8.root')
('Processing', '/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/histos2e2mu_25ns/output_VBF_HToZZTo4L_M125_13TeV_powheg2_JHUgenV6_pythia8.root')
('Processing', '/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/histos2e2mu_25ns/output_ZH_HToZZ_4LFilter_M125_13TeV_powheg2-minlo-HZJ_JHUgenV6_pythia8.root')
('Processing', '/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/histos2e2mu_25ns/output_WminusH_HToZZTo4L_M125_13TeV_powheg2-minlo-HWJ_JHUgenV6_pythia8.root')
('Processing', '/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/histos2e2mu_25ns/output_WplusH_HToZZTo4L_M125_13TeV_powheg2-minlo-HWJ_JHUgenV6_pythia8.root')
('Pr

In [4]:
#clean outliers
krm = []
for ik in events:
    nevents = len(events[ik])
    print '%s events: %i' % (ik,nevents)
    if(nevents < 10):
        krm.append(ik)
        
for ik in krm:
    del events[ik]
keys = []
for ik in events:
    keys.append(ik)
print '----------- KEYS -----------'
print keys    

TTZ events: 183
DYJets events: 0
ZH events: 10135
VBF events: 38623
qqZZ events: 214
WZ events: 2
WH events: 21967
WW events: 0
WJets events: 0
ttH events: 17486
TTbar events: 2
TTWJets events: 3
WZZ events: 1
ggZZ events: 865
WWZ events: 0
GGH events: 16587
ZZZ events: 1
----------- KEYS -----------
['TTZ', 'ZH', 'VBF', 'qqZZ', 'WH', 'ttH', 'ggZZ', 'GGH']


In [5]:
sum = 0
for ik in events:
    sum += len(events[ik])
print 'Total events: %i' % sum

Total events: 106060


In [6]:
#reconstruct Z's and Higgs and organize things
#events = [events2e2mu, events4e, events4mu]
full_event = {}
for ik in keys:
    full_event[ik] = []
    
for ik in keys:
    for iev in range(len(events[ik])):
        #just to recover lepton E to complite p4 - there's uncertanty probably
        lp4 = ROOT.TLorentzVector()
        for il in range(4):
            mass = 0.
            if(abs(events[ik][iev][il][3]) == 13):
                mass = 0.106
            lp4.SetPtEtaPhiM(events[ik][iev][il][0],events[ik][iev][il][1],events[ik][iev][il][2],mass)
            events[ik][iev][il][3] = lp4.E()

        full_event[ik].append([
            events[ik][iev][0],  #l1
            events[ik][iev][1],  #l2
            events[ik][iev][2],  #l3
            events[ik][iev][3],  #l4
            events[ik][iev][4],  #j1
            events[ik][iev][5],  #j2
            events[ik][iev][6],  #j3
            events[ik][iev][7],  #Djet
            events[ik][iev][8],  #MELA
            events[ik][iev][9],  #event weight                
            events[ik][iev][10], #Higgs mass
            events[ik][iev][11], #Z1 mass
            events[ik][iev][12]  #Z2 mass
        ])

In [7]:
# Run classifier with cross-validation and plot ROC curves
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from scipy import interp

from keras.layers import LSTM, Bidirectional, Masking
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.optimizers import SGD, Adam


# Bidirectional LSTM Model
def build_bilstm_model(nparticles, nfeatures):
    # Headline input: meant to receive sequences of 200 floats
    # Note that we can name any layer by passing it a "name" argument.
    i = Input(shape=(nparticles, nfeatures,), name='main_input')
    # the masking layer will prevent the LSTM layer to consider the 0-padded jet values
    m = Masking()(i)

    # A LSTM will transform the vector sequence into a single vector,
    # containing information about the entire sequence
    # the Bidirectional will make the LSTM cell read the sequence from end to start and start to end at the same time
    m = Bidirectional( LSTM(5) ) (m)

    
    auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(m)
    model = Model(input=[i], output=[auxiliary_output])
    opt = Adam()
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model  



# DNN Model
def build_dnn_model(nparticles, nfeatures):
    model = Sequential()
    model.add(Dense(30, input_shape=(nparticles*nfeatures,), activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    opt = Adam()
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model           

In [8]:
nsig = 0
nbkg = 0
for ik in keys:
    if(ik == 'VBF'):
        nsig = len(full_event['VBF'])
    else:
        nbkg += len(full_event[ik])
        
print 'NSig: %i' % nsig
print 'NBkg: %i' % nbkg

#to select how many jets to use 7: 4leptons + 3jets
nparticles = 4
features = {
    'pt' : None,
    'eta': None
    #'phi': None,
    #'e'  : None
}
nfeatures = len(features)
lstm_model = build_bilstm_model(nparticles, nfeatures)
print '\n------------- LSTM MODEL --------------'
print lstm_model.summary()
dnn_model = build_bilstm_model(nparticles, nfeatures)
print '\n------------- DNN MODEL --------------'
print dnn_model.summary()



momentum_input1 = {}
momentum_input1['signal'] = np.zeros((nsig, nparticles, nfeatures)) # 3 momentum
momentum_input1['background'] = np.zeros((nbkg, nparticles, nfeatures)) # 3 momentum

siev = 0
biev = 0
for ik in keys:
    if(ik == 'VBF'):
        for i in range(len(full_event[ik])):
            for j in range(nparticles):
                for iprop in range(nfeatures):
                    if(full_event[ik][i][j][iprop] != -999.):
                        momentum_input1['signal'][siev][j][iprop] = full_event[ik][i][j][iprop]
                    else:
                        break
            siev += 1
    else:
        for i in range(len(full_event[ik])):
            for j in range(nparticles):
                for iprop in range(nfeatures):
                    if(full_event[ik][i][j][iprop] != -999.):
                        momentum_input1['background'][biev][j][iprop] = full_event[ik][i][j][iprop]
                    else:
                        break
            biev += 1
            
            
momentum_input2 = {}
momentum_input2['signal'] = np.zeros((nsig, nparticles*nfeatures)) # 3 momentum
momentum_input2['background'] = np.zeros((nbkg, nparticles*nfeatures)) # 3 momentum

siev = 0
biev = 0
for ik in keys:
    if(ik == 'VBF'):
        for i in range(len(full_event[ik])):
            for j in range(nparticles):
                for iprop in range(nfeatures):
                    if(full_event[ik][i][j][iprop] != -999.):
                        momentum_input2['signal'][siev][j*nfeatures+iprop] = full_event[ik][i][j][iprop]
                    else:
                        break
            siev += 1
    else:
        for i in range(len(full_event[ik])):
            for j in range(nparticles):
                for iprop in range(nfeatures):
                    if(full_event[ik][i][j][iprop] != -999.):
                        momentum_input2['background'][biev][j*nfeatures+iprop] = full_event[ik][i][j][iprop]
                    else:
                        break
            biev += 1
            
print 'siev: %i' % siev
print 'biev: %i' % biev

NSig: 38623
NBkg: 67437

------------- LSTM MODEL --------------
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 4, 2)              0         
_________________________________________________________________
masking_1 (Masking)          (None, 4, 2)              0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 10)                320       
_________________________________________________________________
aux_output (Dense)           (None, 1)                 11        
Total params: 331
Trainable params: 331
Non-trainable params: 0
_________________________________________________________________
None

------------- DNN MODEL --------------
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None

/afs/cern.ch/user/m/mmelodea/keras/lib/python2.7/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[sigmoid.0..., inputs=[/main_inp...)`


In [9]:
#train the LSTM network
fig = pyp.figure(figsize=(10,10))
pyp.rc("font", size=20)

X = np.concatenate([momentum_input1['signal'], momentum_input1['background']])
Y_TT = np.ones(momentum_input1['signal'].shape[0])
Y_QCD = np.zeros(momentum_input1['background'].shape[0])
Y = np.concatenate([Y_TT, Y_QCD])

encoder = LabelEncoder()
encoder.fit(Y)
Y_truth = encoder.transform(Y)

kfold = StratifiedKFold(n_splits=5, shuffle=True,  random_state=seed)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)
colors = cycle(['cyan', 'blue', 'red', 'yellow', 'green', 'orange', 'violet', 'pink', 'brown', 'black'])
lw = 2

i = 0
histories = []
for i,((train, test), color) in enumerate(zip(kfold.split(X, Y_truth), colors)):
    print "\t\tFold",i
    lstm_model = build_bilstm_model(nparticles, nfeatures)
    history = lstm_model.fit(X[train], Y_truth[train], 
                               validation_data=(X[test], Y_truth[test]), 
                               epochs=100, batch_size=128, 
                               verbose=2, callbacks=[early_stopping])
    Y_score = lstm_model.predict(X[test])
    histories.append(history)
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(Y_truth[test], Y_score)
    mean_tpr += interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    roc_auc = auc(fpr, tpr)
    print 'ROC AUC: %.2f' % roc_auc
    pyp.plot(fpr, tpr, lw=lw, color=color, label='ROC fold %d (area = %0.2f)' % (i, roc_auc))

pyp.plot([0, 1], [0, 1], linestyle='--', lw=lw, color='k', label='Luck')
mean_tpr /= kfold.get_n_splits(X, Y_truth)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
print 'ROC AUC: %.2f' % mean_auc

KeyboardInterrupt: 

		Fold 0


/afs/cern.ch/user/m/mmelodea/keras/lib/python2.7/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[sigmoid.0..., inputs=[/main_inp...)`


In [ ]:
pyp.xlim([0, 1.0])
pyp.ylim([0, 1.0])
pyp.xlabel('False Positive Rate')
pyp.ylabel('True Positive Rate')
pyp.title('Receiver Operating Characteristic')
pyp.legend(loc="lower right")
pyp.grid(True)
pyp.show()

In [ ]:
fig = pyp.figure(figsize=(10,10))
#pyp.rc("font", size=20)

for i in range(len(histories)):
    val_loss = np.asarray(histories[i].history['val_loss'])
    loss = np.asarray(histories[i].history['loss'])
    pyp.plot(val_loss, label='loss validation 1')
    pyp.plot(loss, label='loss train 1')
    val_acc = np.asarray(histories[i].history['val_acc'])
    acc = np.asarray(histories[i].history['acc'])
    pyp.plot(val_acc, label='acc validation 1')
    pyp.plot(acc, label='acc train 1')

pyp.legend()
pyp.xlabel('epoch')
pyp.ylim([0.3,0.9])
pyp.show()

In [ ]:
#train the DNN network
fig = pyp.figure(figsize=(10,10))

X = np.concatenate([momentum_input2['signal'], momentum_input2['background']])
Y_TT = np.ones(momentum_input2['signal'].shape[0])
Y_QCD = np.zeros(momentum_input2['background'].shape[0])
Y = np.concatenate([Y_TT, Y_QCD])

encoder = LabelEncoder()
encoder.fit(Y)
Y_truth = encoder.transform(Y)

kfold = StratifiedKFold(n_splits=5, shuffle=True,  random_state=seed)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)
colors = cycle(['cyan', 'blue', 'red', 'yellow', 'green', 'orange', 'violet', 'pink', 'brown', 'black'])
lw = 2

i = 0
histories = []
for i,((train, test), color) in enumerate(zip(kfold.split(X, Y_truth), colors)):
    print "\t\tFold",i
    dnn_model = build_dnn_model(nparticles, nfeatures)
    history = dnn_model.fit(X[train], Y_truth[train], 
                               validation_data=(X[test], Y_truth[test]), 
                               epochs=100, batch_size=128, 
                               verbose=2, callbacks=[early_stopping])
    Y_score = dnn_model.predict(X[test])
    histories.append(history)
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(Y_truth[test], Y_score)
    mean_tpr += interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    roc_auc = auc(fpr, tpr)
    print 'ROC AUC: %.2f' % roc_auc
    pyp.plot(fpr, tpr, lw=lw, color=color, label='ROC fold %d (area = %0.2f)' % (i, roc_auc))

pyp.plot([0, 1], [0, 1], linestyle='--', lw=lw, color='k', label='Luck')
mean_tpr /= kfold.get_n_splits(X, Y_truth)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
print 'ROC AUC: %.2f' % mean_auc

In [ ]:
#fig = pyp.figure(figsize=(10,10))
#pyp.rc("font", size=20)
pyp.xlim([0, 1.0])
pyp.ylim([0, 1.0])
pyp.xlabel('False Positive Rate')
pyp.ylabel('True Positive Rate')
pyp.title('Receiver Operating Characteristic')
pyp.legend(loc="lower right")
pyp.grid(True)
pyp.show()

In [ ]:
fig = pyp.figure(figsize=(10,10))
#pyp.rc("font", size=20)

for i in range(len(histories)):
    val_loss = np.asarray(histories[i].history['val_loss'])
    loss = np.asarray(histories[i].history['loss'])
    pyp.plot(val_loss, label='loss validation 1')
    pyp.plot(loss, label='loss train 1')
    val_acc = np.asarray(histories[i].history['val_acc'])
    acc = np.asarray(histories[i].history['acc'])
    pyp.plot(val_acc, label='acc validation 1')
    pyp.plot(acc, label='acc train 1')

pyp.legend()
pyp.xlabel('epoch')
pyp.ylim([0.3,0.9])
pyp.show()

In [ ]:
#fig = pyp.figure(figsize=(10,10))
pyp.plot(fpr, tpr, lw=lw, color=color, label=('Test ROC (area = %0.2f)' % roc_auc))
pyp.xlim([0, 1.0])
pyp.ylim([0, 1.0])
pyp.xlabel('False Positive Rate')
pyp.ylabel('True Positive Rate')
pyp.title('Receiver Operating Characteristic')
pyp.legend(loc="lower right")
pyp.grid(True)
pyp.show()

In [ ]:
#test over all events and splits per component
fig = pyp.figure(figsize=(10,10))
momentum_input2 = {}
for ik in keys:
    momentum_input2[ik] = np.zeros((len(full_event[ik]), nparticles, nfeatures))

for ik in keys:
    for i in range(len(full_event[ik])):
        for j in range(nparticles):
            for iprop in range(nfeatures):
                if(full_event[ik][i][j][iprop] != -999.):
                    momentum_input2[ik][i][j][iprop] = full_event[ik][i][j][iprop]
                else:
                    break
    
colors = ['black','blue','violet','green','orange','cyan','pink','yellow','red','gray','brown']
global_truth = []
global_score = []
Y_truth = {}
Y_score = {}
for ik in keys:
    X = momentum_input2[ik]
    Y = []
    if(ik == 'VBF'):
        Y = [1 for i in range(len(momentum_input2[ik]))]
    else:
        Y = [0 for i in range(len(momentum_input2[ik]))]
    Y_truth[ik] = Y
    Y_score[ik] = lstm_model.predict(X)
    for i in range(len(Y_truth[ik])):
        global_truth.append(Y_truth[ik][i])
        global_score.append(Y_score[ik][i])
    
#print global_truth
#print global_score

i = 1
for ik in keys:
    if(ik == 'VBF'):
        continue
    biY_truth = np.concatenate([Y_truth['VBF'],Y_truth[ik]])
    biY_score = np.concatenate([Y_score['VBF'],Y_score[ik]])
    fpr, tpr, thresholds = roc_curve(biY_truth, biY_score)
    roc_auc = auc(fpr, tpr)
    print 'VBF vs ',ik,' -- ROC AUC: %.2f' % roc_auc
    pyp.plot(fpr, tpr, lw=2, color=colors[i], label=('VBF vs %s (area = %0.2f)' % (ik,roc_auc)))
    i += 1
    
fpr, tpr, thresholds = roc_curve(global_truth, global_score)
roc_auc = auc(fpr, tpr)
pyp.plot(fpr, tpr, lw=2, color=colors[0], label=('VBF vs BKG (area = %0.2f)' % roc_auc))

pyp.xlim([0, 1.0])
pyp.ylim([0, 1.0])
pyp.xlabel('False Positive Rate')
pyp.ylabel('True Positive Rate')
pyp.title('Receiver Operating Characteristic')
pyp.legend(loc="lower right")
pyp.grid(True)
pyp.show()

In [ ]:
#test over all events
X = np.concatenate([momentum_input['signal'], momentum_input['background']])
Y_TT = np.ones(momentum_input['signal'].shape[0])
Y_QCD = np.zeros(momentum_input['background'].shape[0])
Y = np.concatenate([Y_TT, Y_QCD])
Y_truth = Y
Y_score = lstm_model.predict(X)

fpr, tpr, thresholds = roc_curve(Y_truth, Y_score)
roc_auc = auc(fpr, tpr)
print 'ROC AUC: %.2f' % roc_auc

fig = pyp.figure(figsize=(10,10))
pyp.plot(fpr, tpr, lw=2, color='black', label=('LSTM ROC (area = %0.2f)' % roc_auc))

In [ ]:
full_event2 = {}
full_event2['signal'] = []
full_event2['background'] = []
ik = ''
for ik in keys:
    if(ik == 'VBF'):
        ik2 = 'signal'
    else:
        ik2 = 'background'
    for i in range(len(full_event[ik])):
        full_event2[ik2].append(full_event[ik][i])

In [ ]:
osY = [[full_event2['signal'][i][7]] for i in range(len(full_event2['signal']))]
obY = [[full_event2['background'][i][7]] for i in range(len(full_event2['background']))]
Ydjet = np.concatenate([osY,obY])
print Ydjet

osY = [[full_event2['signal'][i][8]] for i in range(len(full_event2['signal']))]
obY = [[full_event2['background'][i][8]] for i in range(len(full_event2['background']))]
Ymela = np.concatenate([osY,obY])
print Ymela

In [ ]:
fpr, tpr, thresholds = roc_curve(Y_truth, Ydjet)
roc_auc = auc(fpr, tpr)
print 'Djet ROC AUC: %.2f' % roc_auc
pyp.plot(fpr, tpr, lw=2, color='blue', label=('Djet ROC (area = %0.2f)' % roc_auc))

fpr, tpr, thresholds = roc_curve(Y_truth, Ymela)
roc_auc = auc(fpr, tpr)
print 'MELA ROC AUC: %.2f' % roc_auc
pyp.plot(fpr, tpr, lw=2, color='red', label=('MELA ROC (area = %0.2f)' % roc_auc))

pyp.xlim([0, 1.0])
pyp.ylim([0, 1.0])
pyp.xlabel('False Positive Rate')
pyp.ylabel('True Positive Rate')
pyp.title('Receiver Operating Characteristic')
pyp.legend(loc="lower right")
pyp.grid(True)
pyp.show()

In [ ]:
#plot S/B ratio vs S eff
signal_sumw = 0
for i in range(len(full_event2['signal'])):
    signal_sumw += full_event2['signal'][i][9]
print 'Normalized sig: %.2f' % signal_sumw

background_sumw = 0
for i in range(len(full_event2['background'])):
    background_sumw += full_event2['background'][i][9]
print 'Normalized bkg: %.2f' % background_sumw

all_events = np.concatenate([full_event2['signal'], full_event2['background']])

djet_sb = []
djet_seff = []
mela_sb = []
mela_seff = []
lstm_sb = []
lstm_seff = []
djet_cuts = []
mela_cuts = []
lstm_cuts = []

full_sdjet = []
full_bdjet = []
full_smela = []
full_bmela = []
full_slstm = []
full_blstm = []
sweights = []
bweights = []

for cut in range(100):
    icut = cut/100.
    s_djet = 0
    s_mela = 0
    s_lstm = 0
    b_djet = 0
    b_mela = 0
    b_lstm = 0
    for i in range(len(Y_truth)):
        if(Y_truth[i] == 1):
            if(cut == 0):
                full_sdjet.append( Ydjet[i][0] )
                full_smela.append( Ymela[i][0] )
                full_slstm.append( Y_score[i][0] )
                sweights.append( all_events[i][9] )
            
            if(Ydjet[i] > icut):
                s_djet += all_events[i][9]
            if(Ymela[i] > icut):
                s_mela += all_events[i][9]
            if(Y_score[i] > icut):
                s_lstm += all_events[i][9]
        else:
            if(cut == 0):
                full_bdjet.append( Ydjet[i][0] )
                full_bmela.append( Ymela[i][0] )
                full_blstm.append( Y_score[i][0] )
                bweights.append( all_events[i][9] )
            
            if(Ydjet[i] > icut):
                b_djet += all_events[i][9]
            if(Ymela[i] > icut):
                b_mela += all_events[i][9]
            if(Y_score[i] > icut):
                b_lstm += all_events[i][9]
    
    if(b_djet > 0):
        djet_sb.append(s_djet/b_djet)
        djet_seff.append(s_djet/signal_sumw)
        djet_cuts.append(icut)
    
    if(b_mela > 0):
        mela_sb.append(s_mela/b_mela)
        mela_seff.append(s_mela/signal_sumw)
        mela_cuts.append(icut)
    
    if(b_lstm > 0):
        lstm_sb.append(s_lstm/b_lstm)
        lstm_seff.append(s_lstm/signal_sumw)
        lstm_cuts.append(icut)

In [ ]:
fig = pyp.figure(figsize=(10,10))
bins = np.linspace(0, 1, 100)
pyp.hist(full_sdjet, bins, weights=sweights, alpha=0.5, color='red', label='s_Djet')
pyp.hist(full_bdjet, bins, weights=bweights, alpha=0.5, color='blue',label='b_Djet')
pyp.xlabel('Djet')
pyp.ylabel('Normalized')
pyp.ylim([0.001,10])
pyp.legend(loc='upper right')
pyp.gca().set_yscale("log")
pyp.show()

In [ ]:
fig = pyp.figure(figsize=(10,10))
bins = np.linspace(0, 1, 100)
pyp.hist(full_smela, bins, weights=sweights, alpha=0.5, color='red', label='s_MELA')
pyp.hist(full_bmela, bins, weights=bweights, alpha=0.5, color='blue',label='b_MELA')
pyp.xlabel('MELA')
pyp.ylabel('Normalized')
pyp.ylim([0.001,10])
pyp.legend(loc='upper right')
pyp.gca().set_yscale("log")
pyp.show()

In [ ]:
fig = pyp.figure(figsize=(10,10))
bins = np.linspace(0, 1, 100)
pyp.hist(full_slstm, bins, weights=sweights, alpha=0.5, color='red', label='s_LSTM')
pyp.hist(full_blstm, bins, weights=bweights, alpha=0.5, color='blue',label='b_LSTM')
pyp.xlabel('LSTM')
pyp.ylabel('Normalized')
pyp.ylim([0.001,10])
pyp.legend(loc='upper right')
pyp.gca().set_yscale("log")
pyp.show()

In [ ]:
fig = pyp.figure(figsize=(10,10))
pyp.plot(djet_seff, djet_sb, lw=2, color='blue', label='Djet')
pyp.plot(mela_seff, mela_sb, lw=2, color='red', label='MELA')
pyp.plot(lstm_seff, lstm_sb, lw=2, color='black', label='LSTM')
pyp.xlim([0, 1])
pyp.xlabel('Seff')
pyp.ylim([0, 10])
pyp.ylabel('S/B')
pyp.legend(loc="upper right")
pyp.grid(True)
pyp.show()

In [ ]:
fig = pyp.figure(figsize=(10,10))
pyp.plot(djet_cuts, djet_sb, lw=2, color='blue', label='Djet')
pyp.plot(mela_cuts, mela_sb, lw=2, color='red', label='MELA')
pyp.plot(lstm_cuts, lstm_sb, lw=2, color='black', label='LSTM')
pyp.xlim([0, 1])
pyp.ylim([0, 10])
pyp.xlabel('Cut')
pyp.ylabel('S/B')
pyp.legend(loc="upper left")
pyp.grid(True)
pyp.show()

In [ ]:
fig = pyp.figure(figsize=(10,10))
pyp.plot(djet_cuts, djet_seff, lw=2, color='blue', label='Djet')
pyp.plot(mela_cuts, mela_seff, lw=2, color='red', label='MELA')
pyp.plot(lstm_cuts, lstm_seff, lw=2, color='black', label='LSTM')
pyp.xlim([0, 1])
pyp.ylim([0, 1])
pyp.xlabel('Cut')
pyp.ylabel('Signal eff')
pyp.legend(loc="upper left")
pyp.grid(True)

pyp.show()

In [ ]:
events = formatInputs(files_addresses, 2, 0, 10000)
print '------- YIELDS -------'
for ik in events:
    nevents = len(events[ik])
    print '%s events: %i' % (ik,nevents)

In [ ]:
#m4l plot
colors = ['black','blue','violet','green','orange','cyan','pink','yellow','red','gray','brown','cyan','pink','yellow','red','gray','brown']
keys = [ik for ik in events]

true_m4l = [[] for i in keys]
wgs = [[] for i in keys]
j = 0
for ik in keys:
    for i in range(len(events[ik])):
            true_m4l[j].append(events[ik][i][10])
            wgs[j].append(events[ik][i][9])
    j += 1

n_bins = 265
fig = pyp.figure(figsize=(10,10))
pyp.hist(true_m4l, n_bins, weights=wgs, stacked=True, color=colors, label=keys)
pyp.xlim([70, 600])
pyp.xlabel('TRUE m4l')
pyp.ylabel('Normalized')
pyp.legend(loc="upper right")
pyp.show()